In [1]:
import os
import sys

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import pandas as pd

from ray import tune
from ray.air import session
from ray.tune.schedulers import ASHAScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pathlib import Path

module_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if module_path not in sys.path:
    os.environ['PYTHONPATH'] = module_path
    sys.path.append(module_path)

from src import train

In [2]:
target_variable = 'Flow_Kalltveit'
file_name = "cleaned_data_1.csv"
data_dir = "../data"
datetime_variable = "Datetime"

In [3]:
config = {
    "data_file": file_name,
    "datetime":  datetime_variable,
    "target_variable": target_variable,
    "arch": tune.grid_search(["LSTM", "LSTMTemporalAttention", "LSTMSpatialTemporalAttention"]), # "FCN", "FCNTemporalAttention", "LSTMTemporalAttention", "LSTM", "LSTMSpatialAttention", "LSTMSpatialTemporalAttention"
    "sequence_length": tune.choice([25]),
    'num_epochs': tune.choice([30]),
    'num_layers': tune.choice([2, 3, 4]),
    "learning_rate": tune.loguniform(1e-4, 1e-1),
    "weigth_decay": tune.choice([0, 0.001, 0.0001]),
    "batch_size": tune.choice([256, 512]),
    "hidden_size": tune.choice([32, 64]),
    "variables": tune.grid_search([
        None,
        ["Wind_Speed_Nilsebu", "Air_Temperature_Nilsebu", "Wind_Direction_Nilsebu", "Relative_Humidity_Nilsebu", 
         "Air_Temperature_Fister", "Precipitation_Fister", "Flow_Lyngsvatn_Overflow", "Flow_Tapping", 
         "Water_Level_Kalltveit", "Water_Temperature_Kalltveit_Kum", "Precipitation_Nilsebu", "Flow_HBV", 
         "Precipitation_HBV", "Temperature_HBV", "Flow_Without_Tapping_Kalltveit", "Flow_Lyngsaana", 
         "Water_Temperature_Lyngsaana"],
    ])
}

scheduler = ASHAScheduler( # TODO: Find a scheduler that works better
    metric="val_loss",
    mode="min",
    max_t=100,
    grace_period=5,
    reduction_factor=2
)

reporter = tune.JupyterNotebookReporter(
        parameter_columns={
            "weigth_decay": "w_decay",
            "learning_rate": "lr",
            "num_epochs": "num_epochs"
        },
        metric_columns=[
            "train_loss", "val_loss", "test_loss", "training_iteration"
        ])

analysis = tune.run(
    train.train_model, # TODO: partial(train_cifar, data_dir=data_dir),
    resources_per_trial={"cpu": 12, "gpu": 1},
    config=config,
    num_samples=1,
    #scheduler=scheduler,
    progress_reporter=reporter,
    name="inflow_forecasting",
    
)#time_total_s require_attrs=False,

2023-04-02 20:50:59,339	INFO worker.py:1553 -- Started a local Ray instance.
c:\Code\Master\hydro-ml\.venv\lib\site-packages\ray\tune\trainable\function_trainable.py:612: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/master/tune/api_docs/trainable.html

  warnings.warn(
  0%|          | 0/23 [00:00<?, ?it/s]


Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,test_loss,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,training_iteration,trial_id,val_loss,warmup_time
train_model_4fc82_00000,2023-04-02_20-51-08,False,,4a331304cfc840728d3638678611d1eb,DESKTOP-D4IVECG,1,127.0.0.1,3292,0,3.79801,3.79801,3.79801,1680461468,0,,60.3569,1,4fc82_00000,49.4293,0.00500584


  0%|          | 0/89 [00:00<?, ?it/s]
(train_model pid=4276) c:\Code\Master\hydro-ml\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([25, 512])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(train_model pid=4276)   return F.mse_loss(input, target, reduction=self.reduction)
 51%|█████     | 45/89 [00:00<00:00, 103.81it/s]
(train_model pid=4276) c:\Code\Master\hydro-ml\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([492])) that is different to the input size (torch.Size([25, 492])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(train_model pid=4276)   return F.mse_loss(input, target, reduction=self.reduction)
  0%|          | 0/23 [00:00<?, ?it/s]c:\Code\Master\hydro-ml\.venv\lib\site-packages\torch\nn\modules\loss.

In [4]:
print("Best config: ", analysis.get_best_config(
   metric="val_loss", mode="min"))
# Get a dataframe for analyzing trial results.
df = analysis.results_df

Best config:  {'data_file': 'cleaned_data_1.csv', 'datetime': 'Datetime', 'target_variable': 'Flow_Kalltveit', 'arch': 'LSTMSpatialTemporalAttention', 'sequence_length': 25, 'num_epochs': 30, 'num_layers': 3, 'learning_rate': 0.0016789584101260435, 'weigth_decay': 0, 'batch_size': 512, 'hidden_size': 64, 'variables': ['Wind_Speed_Nilsebu', 'Air_Temperature_Nilsebu', 'Wind_Direction_Nilsebu', 'Relative_Humidity_Nilsebu', 'Air_Temperature_Fister', 'Precipitation_Fister', 'Flow_Lyngsvatn_Overflow', 'Flow_Tapping', 'Water_Level_Kalltveit', 'Water_Temperature_Kalltveit_Kum', 'Precipitation_Nilsebu', 'Flow_HBV', 'Precipitation_HBV', 'Temperature_HBV', 'Flow_Without_Tapping_Kalltveit', 'Flow_Lyngsaana', 'Water_Temperature_Lyngsaana']}


In [5]:
df[['config/arch', 'train_loss', 'val_loss', 'test_loss', 'config/variables']].sort_values('test_loss')

,config/arch,train_loss,val_loss,test_loss,config/variables
trial_id,,,,,
4fc82_00005,LSTMTemporalAttention,0.548020,0.191651,0.250804,"[Wind_Speed_Nilsebu, Air_Temperature_Nilsebu, ..."
4fc82_00007,LSTMSpatialTemporalAttention,0.884399,0.189990,0.324477,"[Wind_Speed_Nilsebu, Air_Temperature_Nilsebu, ..."
4fc82_00000,LSTM,2.127757,0.600998,0.864389,None
4fc82_00001,LSTMTemporalAttention,1.773299,0.655915,0.976507,None
4fc82_00003,LSTMSpatialTemporalAttention,12.081190,3.553490,4.621995,None
4fc82_00002,LSTMSpatialAttention,27.111710,21.432994,22.001630,None
4fc82_00004,LSTM,58.288598,56.025162,51.285787,"[Wind_Speed_Nilsebu, Air_Temperature_Nilsebu, ..."
4fc82_00006,LSTMSpatialAttention,58.337091,56.076422,51.356407,"[Wind_Speed_Nilsebu, Air_Temperature_Nilsebu, ..."
